# MaterialX Shader Translators

There are a series of BXDF shader translation definitions which are implemented as `nodegraphs`.
They can be found in `libraries/bxdf/translators/`.

They do not map from all shading models to all other shading models, and have been created based on the what is considered to tbe the "standard" shading model and what other shading models this standard model would typically be translated to. Only surface shading models are currently supported.

The current proposed standard is the "Open PBR Surface" model. See this link for more information: [OpenPBR](https://github.com/AcademySoftwareFoundation/OpenPBR). At time of writing however the "Autodesk Standard Surface" model has the most translation support.

## Translators

Translation graphs are not limited purely for mapping BXDF nodes but could be used to map any node to any other node type.
Additionally it would be useful to map from one version of a given node to another version of the same node type. There is currently no support for this but it is a planned future enhancement (at time of writing this is version 1.39.5 or later).



## API Support

The MaterialX API provides support for using these translators via the [ShaderTranslator](https://materialx.org/docs/api/class_shader_translator.html) class.

An example Python script is available as part of the Python package and can be found under `python/Scripts/translateshader.py`.
The script is overly packaged to include texture baking as well which is not a requirement for remapping.

For clarity we pull out only the required setup here.


### Setup

First we need to create a document with the appropriate definition libraries loaded. This will include the translator definitions.

In [507]:
import MaterialX as mx

def creeate_working_document() -> dict[str, mx.Document]:
    doc : mx.Document = mx.createDocument()
    stdlib : mx.Document = mx.createDocument()

    searchPath : mx.FileSearchPath = mx.getDefaultDataSearchPath()
    libraryFolders : list[mx.FilePath]= mx.getDefaultDataLibraryFolders()
    libraryFiles : set[str] = mx.loadLibraries(libraryFolders, searchPath, stdlib)
    doc.setDataLibrary(stdlib)
    nodedefs : list[mx.NodeDef] = doc.getNodeDefs()
    print(f"Created working doc with {len(nodedefs)} nodedefs from standard library.")

    result = { "doc": doc, "stdlib": stdlib }
    return result

result = creeate_working_document()
doc : mx.Document = result["doc"]
stdlib : mx.Document = result["stdlib"]


Created working doc with 803 nodedefs from standard library.


All translators have the `nodegroup` attribute set as `translation`. 

In [508]:

# Look for nodedefs which are translators.
translators : list[mx.NodeDef] = [nd for nd in doc.getNodeDefs() if nd.getNodeGroup() == "translation"]
print(f"Found {len(translators)} translators in the standard library.")
for translator in translators:
    print(f"Translator: {translator.getName()} for target: {translator.getAttribute('target')}")    

Found 4 translators in the standard library.
Translator: ND_open_pbr_surface_to_standard_surface for target: 
Translator: ND_standard_surface_to_gltf_pbr for target: 
Translator: ND_standard_surface_to_open_pbr_surface for target: 
Translator: ND_standard_surface_to_UsdPreviewSurface for target: 


Finding out what is being translated is obfiscated as it's encoded into the name of the node definition (`nodedef`).

For example "OpenPBR to standard surface is defined as `ND_open_pbr_surface_to_standard_surface`. 
- Given a definition to extract the "from" and "to" parts we can split the name at `_to_` and then remove the `ND_` prefix. 
- Given a desired "from" and "to" we need to build the name by inserting `_to_` between the two and adding the `ND_` prefix.

In [509]:
def derive_translator_name_from_targets(source : str, target : str) -> str:
    return f"ND_{source}_to_{target}"

def find_targets_in_translator_name(name : str) -> tuple[str, str] | None:
    prefix = "ND_"
    infix = "_to_"
    if not name.startswith(prefix):
        return None
    parts = name[len(prefix):].split(infix)
    if len(parts) != 2:
        return None
    return (parts[0], parts[1])


In [510]:

# Get source and target from name
targets_list : list[tuple[str, str]] = []
for translator in translators:
    translator_name = translator.getName()
    targets = find_targets_in_translator_name(translator_name)
    if targets:
        targets_list.append(targets)
        print(f"Translator name '{translator_name}' maps source: '{targets[0]}' to target: '{targets[1]}'")


Translator name 'ND_open_pbr_surface_to_standard_surface' maps source: 'open_pbr_surface' to target: 'standard_surface'
Translator name 'ND_standard_surface_to_gltf_pbr' maps source: 'standard_surface' to target: 'gltf_pbr'
Translator name 'ND_standard_surface_to_open_pbr_surface' maps source: 'standard_surface' to target: 'open_pbr_surface'
Translator name 'ND_standard_surface_to_UsdPreviewSurface' maps source: 'standard_surface' to target: 'UsdPreviewSurface'


In [511]:
def find_translator(doc : mx.Document, source : str, target : str) -> mx.NodeDef:
    derived_name = derive_translator_name_from_targets(source, target)
    # Look for the translator in the document
    translator_nodedef : mx.NodeDef = doc.getNodeDef(derived_name)
    return translator_nodedef

# Derive name from source and target
for source, target in targets_list:
    translator_nodedef : mx.NodeDef = find_translator(doc, source, target)
    if translator_nodedef:
        print(f"- Found translator nodedef: '{translator_nodedef.getName()}' in document.")

- Found translator nodedef: 'ND_open_pbr_surface_to_standard_surface' in document.
- Found translator nodedef: 'ND_standard_surface_to_gltf_pbr' in document.
- Found translator nodedef: 'ND_standard_surface_to_open_pbr_surface' in document.
- Found translator nodedef: 'ND_standard_surface_to_UsdPreviewSurface' in document.


There is no clean API for finding BXDF shading models as they have not specific classification attribute.
Below is a utility to find this based on existing library information. 

The `NodeDef.getNodeString()` method
provides the name of shading models that can as input to search for translators.

In [512]:
# Scan all nodedefs with output type of "surfaceshader"
def find_all_bxdf(doc : mx.Document) -> list[mx.NodeDef]:
    bxdfs : list[mx.NodeDef] = []
    for nodedef in doc.getNodeDefs():
        if nodedef.getType() == "surfaceshader":    
            if nodedef.getNodeString() not in ["convert", "surface"] and nodedef.getNodeGroup() == "pbr":   
                bxdfs.append(nodedef)
    return bxdfs

bdxfs = find_all_bxdf(doc)
print(f"Found {len(bdxfs)} shading models in the document:")
for bxdf in bdxfs:
    print(f"- Model: NodeDef identifier {bxdf.getName()}. Classification: {bxdf.getNodeString()} ")

Found 6 shading models in the document:
- Model: NodeDef identifier ND_disney_principled. Classification: disney_principled 
- Model: NodeDef identifier ND_gltf_pbr_surfaceshader. Classification: gltf_pbr 
- Model: NodeDef identifier ND_open_pbr_surface_surfaceshader. Classification: open_pbr_surface 
- Model: NodeDef identifier ND_standard_surface_surfaceshader. Classification: standard_surface 
- Model: NodeDef identifier ND_standard_surface_surfaceshader_100. Classification: standard_surface 
- Model: NodeDef identifier ND_UsdPreviewSurface_surfaceshader. Classification: UsdPreviewSurface 


In [513]:
# Create one of each shading model node

for node in doc.getNodes():
    doc.removeNode(node.getName())

def create_bxdf_node(doc : mx.Document, bxdf, add_all_inputs=False) -> mx.Node:

    bxdf_node : mx.Node = doc.addNodeInstance(bxdf, "test_" + bxdf.getName())
    nodedef = doc.getNodeDef(bxdf.getName())
    if add_all_inputs:
        bxdf_node.addInputsFromNodeDef()
    for input in bxdf_node.getInputs():
        if not input.getValue():
            nodedef_input : mx.Input = nodedef.getInput(input.getName())    
            added_default = False
            #if nodedef_input:
            #    default_geom_prop = nodedef_input.getDefaultGeomPropString()
            #    if default_geom_prop:
            #        input.setDefaultGeomPropString(default_geom_prop)
            #        added_default = True
            if not added_default:
                #input.setDefaultValue(noddef.getInput(input.getName()).getDefaultValue())
                bxdf_node.removeInput(input.getName())
    return bxdf_node

for bxdf in bdxfs:
    #print('Creating instace of nodedef:', nodedef.getName())
     bxdf_node : mx.Node = create_bxdf_node(doc, bxdf)
     if bxdf_node:
         print(f"- Created node instance of '{bxdf.getName()}' with {len(bxdf_node.getInputs())} inputs.")  

valid, error = doc.validate()
print(f"- Document validation: {valid}. Errors: '{error}'")


from IPython.display import display_markdown
def print_doc(doc : mx.Document):
    doc_string : str = mx.writeToXmlString(doc)
    display_markdown('```xml\n' + doc_string + '\n```\n', raw=True)
print_doc(doc)


- Created node instance of 'ND_disney_principled' with 0 inputs.
- Created node instance of 'ND_gltf_pbr_surfaceshader' with 0 inputs.
- Created node instance of 'ND_open_pbr_surface_surfaceshader' with 0 inputs.
- Created node instance of 'ND_standard_surface_surfaceshader' with 0 inputs.
- Created node instance of 'ND_standard_surface_surfaceshader_100' with 0 inputs.
- Created node instance of 'ND_UsdPreviewSurface_surfaceshader' with 0 inputs.
- Document validation: True. Errors: ''


```xml
<?xml version="1.0"?>
<materialx version="1.39">
  <disney_principled name="test_ND_disney_principled" type="surfaceshader" nodedef="ND_disney_principled" />
  <gltf_pbr name="test_ND_gltf_pbr_surfaceshader" type="surfaceshader" nodedef="ND_gltf_pbr_surfaceshader" />
  <open_pbr_surface name="test_ND_open_pbr_surface_surfaceshader" type="surfaceshader" nodedef="ND_open_pbr_surface_surfaceshader" />
  <standard_surface name="test_ND_standard_surface_surfaceshader" type="surfaceshader" nodedef="ND_standard_surface_surfaceshader" />
  <standard_surface name="test_ND_standard_surface_surfaceshader_100" type="surfaceshader" nodedef="ND_standard_surface_surfaceshader_100" />
  <UsdPreviewSurface name="test_ND_UsdPreviewSurface_surfaceshader" type="surfaceshader" nodedef="ND_UsdPreviewSurface_surfaceshader" />
</materialx>

```


There are two exposed interfaces on the `ShaderTranslator` class:
- `translateShader()`
- `translateAllMaterials()`

The first translates a given **shader** to the target shading model.
The second translates all **materials** in the document to the target shading model. In general this one is mostly useless
as it halts if there are any failures, and there is no way to decide which materials to translate.

In [514]:
from MaterialX import PyMaterialXGenShader as mx_gen_shader
translator : mx_gen_shader.ShaderTranslator = mx_gen_shader.ShaderTranslator.create()
try:
    translator.translateAllMaterials(doc, 'standard_surface')
except LookupError as err:
    print(err)
        

Instead we find all BXDF shader nodes in the document explicitly.

In [515]:
def get_surface_shader_nodes(doc : mx.Document) -> list[mx.Node]:
    surface_shader_nodes : list[mx.Node] = []
    for node in doc.getNodes():
        nodedef : mx.NodeDef | None = node.getNodeDef()
        if nodedef and nodedef.getType() == "surfaceshader":
            surface_shader_nodes.append(node)
    return surface_shader_nodes

surface_shader_nodes = get_surface_shader_nodes(doc)
print(f"Found {len(surface_shader_nodes)} surface shader nodes in the document.")
for node in surface_shader_nodes:
    print(f"- Surface shader node: '{node.getName()}' of type '{node.getType()}' with nodedef '{node.getNodeDef().getName()}'") 

Found 6 surface shader nodes in the document.
- Surface shader node: 'test_ND_disney_principled' of type 'surfaceshader' with nodedef 'ND_disney_principled'
- Surface shader node: 'test_ND_gltf_pbr_surfaceshader' of type 'surfaceshader' with nodedef 'ND_gltf_pbr_surfaceshader'
- Surface shader node: 'test_ND_open_pbr_surface_surfaceshader' of type 'surfaceshader' with nodedef 'ND_open_pbr_surface_surfaceshader'
- Surface shader node: 'test_ND_standard_surface_surfaceshader' of type 'surfaceshader' with nodedef 'ND_standard_surface_surfaceshader'
- Surface shader node: 'test_ND_standard_surface_surfaceshader_100' of type 'surfaceshader' with nodedef 'ND_standard_surface_surfaceshader_100'
- Surface shader node: 'test_ND_UsdPreviewSurface_surfaceshader' of type 'surfaceshader' with nodedef 'ND_UsdPreviewSurface_surfaceshader'


Then we translate them one at a time.

Note that if a translate is not found it has a negative side-effect of leaving behind empty node graphs (`nodegraph`) elements.
We remove them here as a post cleanup step for clarity.

In [516]:
from MaterialX import PyMaterialXGenShader as mx_gen_shader
successfull_nodes : list[mx.Node] = []
for node in surface_shader_nodes:
    target_bxdf = "standard_surface"
    translator = mx_gen_shader.ShaderTranslator.create()
    try:
        source_bxdf = node.getCategory()
        nodedef : mx.NodeDef | None = find_translator(doc, source_bxdf, target_bxdf)
        if not nodedef:
            # Throw an exception to be caught below
            raise LookupError(f"No translator found from '{source_bxdf}' to '{target_bxdf}' for node '{node.getName()}'")
        translator.translateShader(node, target_bxdf )
        successfull_nodes.append(node)
        print(f"- Translated node '{node.getName()}' to target '{target_bxdf}'")
    except LookupError as err:
        print(f"- Failed to translate node '{node.getName()}' to target '{target_bxdf}': {err}")

# Cleanup empty nodegraphs
for nodegraph in doc.getNodeGraphs():
    if len(nodegraph.getNodes()) == 0:
        print('- Removing empty nodegraph:', nodegraph.getName())
        doc.removeNodeGraph(nodegraph.getName())
#print_doc(doc)

filepath : mx.FilePath = mx.FilePath("data/translated_materials.mtlx")
print("- Writing translated document to 'data/translated_materials.mtlx'")
mx.writeToXmlFile(doc, filename=filepath)


- Failed to translate node 'test_ND_disney_principled' to target 'standard_surface': No translator found from 'disney_principled' to 'standard_surface' for node 'test_ND_disney_principled'
- Failed to translate node 'test_ND_gltf_pbr_surfaceshader' to target 'standard_surface': No translator found from 'gltf_pbr' to 'standard_surface' for node 'test_ND_gltf_pbr_surfaceshader'
- Translated node 'test_ND_open_pbr_surface_surfaceshader' to target 'standard_surface'
- Failed to translate node 'test_ND_standard_surface_surfaceshader' to target 'standard_surface': No translator found from 'standard_surface' to 'standard_surface' for node 'test_ND_standard_surface_surfaceshader'
- Failed to translate node 'test_ND_standard_surface_surfaceshader_100' to target 'standard_surface': No translator found from 'standard_surface' to 'standard_surface' for node 'test_ND_standard_surface_surfaceshader_100'
- Failed to translate node 'test_ND_UsdPreviewSurface_surfaceshader' to target 'standard_surface'

Below is the result shown as a Mermaid diagram:
```mermaid
graph LR
    test_ND_disney_principled[disney_principled:surfaceshader]
    test_ND_gltf_pbr_surfaceshader[gltf_pbr:surfaceshader]
    test_ND_open_pbr_surface_surfaceshader[standard_surface:surfaceshader]
    test_ND_standard_surface_surfaceshader[standard_surface:surfaceshader]
    test_ND_standard_surface_surfaceshader_100[standard_surface:surfaceshader]
    test_ND_UsdPreviewSurface_surfaceshader[UsdPreviewSurface:surfaceshader]
    subgraph nodegraph3
    nodegraph3_base_out([output:float])
    style nodegraph3_base_out  fill:#09D, color:#FFF
    nodegraph3_base_color_out([output:color3])
    style nodegraph3_base_color_out  fill:#09D, color:#FFF
    nodegraph3_diffuse_roughness_out([output:float])
    style nodegraph3_diffuse_roughness_out  fill:#09D, color:#FFF
    nodegraph3_metalness_out([output:float])
    style nodegraph3_metalness_out  fill:#09D, color:#FFF
    nodegraph3_specular_out([output:float])
    style nodegraph3_specular_out  fill:#09D, color:#FFF
    nodegraph3_specular_color_out([output:color3])
    style nodegraph3_specular_color_out  fill:#09D, color:#FFF
    nodegraph3_specular_roughness_out([output:float])
    style nodegraph3_specular_roughness_out  fill:#09D, color:#FFF
    nodegraph3_specular_IOR_out([output:float])
    style nodegraph3_specular_IOR_out  fill:#09D, color:#FFF
    nodegraph3_specular_anisotropy_out([output:float])
    style nodegraph3_specular_anisotropy_out  fill:#09D, color:#FFF
    nodegraph3_transmission_out([output:float])
    style nodegraph3_transmission_out  fill:#09D, color:#FFF
    nodegraph3_transmission_color_out([output:color3])
    style nodegraph3_transmission_color_out  fill:#09D, color:#FFF
    nodegraph3_transmission_depth_out([output:float])
    style nodegraph3_transmission_depth_out  fill:#09D, color:#FFF
    nodegraph3_transmission_scatter_out([output:color3])
    style nodegraph3_transmission_scatter_out  fill:#09D, color:#FFF
    nodegraph3_transmission_scatter_anisotropy_out([output:float])
    style nodegraph3_transmission_scatter_anisotropy_out  fill:#09D, color:#FFF
    nodegraph3_transmission_dispersion_out([output:float])
    style nodegraph3_transmission_dispersion_out  fill:#09D, color:#FFF
    nodegraph3_subsurface_out([output:float])
    style nodegraph3_subsurface_out  fill:#09D, color:#FFF
    nodegraph3_subsurface_color_out([output:color3])
    style nodegraph3_subsurface_color_out  fill:#09D, color:#FFF
    nodegraph3_subsurface_radius_out([output:color3])
    style nodegraph3_subsurface_radius_out  fill:#09D, color:#FFF
    nodegraph3_subsurface_scale_out([output:float])
    style nodegraph3_subsurface_scale_out  fill:#09D, color:#FFF
    nodegraph3_subsurface_anisotropy_out([output:float])
    style nodegraph3_subsurface_anisotropy_out  fill:#09D, color:#FFF
    nodegraph3_sheen_out([output:float])
    style nodegraph3_sheen_out  fill:#09D, color:#FFF
    nodegraph3_sheen_color_out([output:color3])
    style nodegraph3_sheen_color_out  fill:#09D, color:#FFF
    nodegraph3_sheen_roughness_out([output:float])
    style nodegraph3_sheen_roughness_out  fill:#09D, color:#FFF
    nodegraph3_coat_out([output:float])
    style nodegraph3_coat_out  fill:#09D, color:#FFF
    nodegraph3_coat_color_out([output:color3])
    style nodegraph3_coat_color_out  fill:#09D, color:#FFF
    nodegraph3_coat_roughness_out([output:float])
    style nodegraph3_coat_roughness_out  fill:#09D, color:#FFF
    nodegraph3_coat_anisotropy_out([output:float])
    style nodegraph3_coat_anisotropy_out  fill:#09D, color:#FFF
    nodegraph3_coat_IOR_out([output:float])
    style nodegraph3_coat_IOR_out  fill:#09D, color:#FFF
    nodegraph3_coat_affect_roughness_out([output:float])
    style nodegraph3_coat_affect_roughness_out  fill:#09D, color:#FFF
    nodegraph3_thin_film_thickness_out([output:float])
    style nodegraph3_thin_film_thickness_out  fill:#09D, color:#FFF
    nodegraph3_thin_film_IOR_out([output:float])
    style nodegraph3_thin_film_IOR_out  fill:#09D, color:#FFF
    nodegraph3_emission_out([output:float])
    style nodegraph3_emission_out  fill:#09D, color:#FFF
    nodegraph3_emission_color_out([output:color3])
    style nodegraph3_emission_color_out  fill:#09D, color:#FFF
    nodegraph3_opacity_out([output:color3])
    style nodegraph3_opacity_out  fill:#09D, color:#FFF
    nodegraph3_thin_walled_out([output:boolean])
    style nodegraph3_thin_walled_out  fill:#09D, color:#FFF
    nodegraph3_node1[open_pbr_surface_to_standard_surface:multioutput]
    end
    nodegraph3_base_out --"base"--> test_ND_open_pbr_surface_surfaceshader
    nodegraph3_base_color_out --"base_color"--> test_ND_open_pbr_surface_surfaceshader
    nodegraph3_diffuse_roughness_out --"diffuse_roughness"--> test_ND_open_pbr_surface_surfaceshader
    nodegraph3_metalness_out --"metalness"--> test_ND_open_pbr_surface_surfaceshader
    nodegraph3_specular_out --"specular"--> test_ND_open_pbr_surface_surfaceshader
    nodegraph3_specular_color_out --"specular_color"--> test_ND_open_pbr_surface_surfaceshader
    nodegraph3_specular_roughness_out --"specular_roughness"--> test_ND_open_pbr_surface_surfaceshader
    nodegraph3_specular_IOR_out --"specular_IOR"--> test_ND_open_pbr_surface_surfaceshader
    nodegraph3_specular_anisotropy_out --"specular_anisotropy"--> test_ND_open_pbr_surface_surfaceshader
    nodegraph3_transmission_out --"transmission"--> test_ND_open_pbr_surface_surfaceshader
    nodegraph3_transmission_color_out --"transmission_color"--> test_ND_open_pbr_surface_surfaceshader
    nodegraph3_transmission_depth_out --"transmission_depth"--> test_ND_open_pbr_surface_surfaceshader
    nodegraph3_transmission_scatter_out --"transmission_scatter"--> test_ND_open_pbr_surface_surfaceshader
    nodegraph3_transmission_scatter_anisotropy_out --"transmission_scatter_anisotropy"--> test_ND_open_pbr_surface_surfaceshader
    nodegraph3_transmission_dispersion_out --"transmission_dispersion"--> test_ND_open_pbr_surface_surfaceshader
    nodegraph3_subsurface_out --"subsurface"--> test_ND_open_pbr_surface_surfaceshader
    nodegraph3_subsurface_color_out --"subsurface_color"--> test_ND_open_pbr_surface_surfaceshader
    nodegraph3_subsurface_radius_out --"subsurface_radius"--> test_ND_open_pbr_surface_surfaceshader
    nodegraph3_subsurface_scale_out --"subsurface_scale"--> test_ND_open_pbr_surface_surfaceshader
    nodegraph3_subsurface_anisotropy_out --"subsurface_anisotropy"--> test_ND_open_pbr_surface_surfaceshader
    nodegraph3_sheen_out --"sheen"--> test_ND_open_pbr_surface_surfaceshader
    nodegraph3_sheen_color_out --"sheen_color"--> test_ND_open_pbr_surface_surfaceshader
    nodegraph3_sheen_roughness_out --"sheen_roughness"--> test_ND_open_pbr_surface_surfaceshader
    nodegraph3_coat_out --"coat"--> test_ND_open_pbr_surface_surfaceshader
    nodegraph3_coat_color_out --"coat_color"--> test_ND_open_pbr_surface_surfaceshader
    nodegraph3_coat_roughness_out --"coat_roughness"--> test_ND_open_pbr_surface_surfaceshader
    nodegraph3_coat_anisotropy_out --"coat_anisotropy"--> test_ND_open_pbr_surface_surfaceshader
    nodegraph3_coat_IOR_out --"coat_IOR"--> test_ND_open_pbr_surface_surfaceshader
    nodegraph3_coat_affect_roughness_out --"coat_affect_roughness"--> test_ND_open_pbr_surface_surfaceshader
    nodegraph3_thin_film_thickness_out --"thin_film_thickness"--> test_ND_open_pbr_surface_surfaceshader
    nodegraph3_thin_film_IOR_out --"thin_film_IOR"--> test_ND_open_pbr_surface_surfaceshader
    nodegraph3_emission_out --"emission"--> test_ND_open_pbr_surface_surfaceshader
    nodegraph3_emission_color_out --"emission_color"--> test_ND_open_pbr_surface_surfaceshader
    nodegraph3_opacity_out --"opacity"--> test_ND_open_pbr_surface_surfaceshader
    nodegraph3_thin_walled_out --"thin_walled"--> test_ND_open_pbr_surface_surfaceshader
    nodegraph3_node1 --"base_out"--> nodegraph3_base_out
    nodegraph3_node1 --"base_color_out"--> nodegraph3_base_color_out
    nodegraph3_node1 --"diffuse_roughness_out"--> nodegraph3_diffuse_roughness_out
    nodegraph3_node1 --"metalness_out"--> nodegraph3_metalness_out
    nodegraph3_node1 --"specular_out"--> nodegraph3_specular_out
    nodegraph3_node1 --"specular_color_out"--> nodegraph3_specular_color_out
    nodegraph3_node1 --"specular_roughness_out"--> nodegraph3_specular_roughness_out
    nodegraph3_node1 --"specular_IOR_out"--> nodegraph3_specular_IOR_out
    nodegraph3_node1 --"specular_anisotropy_out"--> nodegraph3_specular_anisotropy_out
    nodegraph3_node1 --"transmission_out"--> nodegraph3_transmission_out
    nodegraph3_node1 --"transmission_color_out"--> nodegraph3_transmission_color_out
    nodegraph3_node1 --"transmission_depth_out"--> nodegraph3_transmission_depth_out
    nodegraph3_node1 --"transmission_scatter_out"--> nodegraph3_transmission_scatter_out
    nodegraph3_node1 --"transmission_scatter_anisotropy_out"--> nodegraph3_transmission_scatter_anisotropy_out
    nodegraph3_node1 --"transmission_dispersion_out"--> nodegraph3_transmission_dispersion_out
    nodegraph3_node1 --"subsurface_out"--> nodegraph3_subsurface_out
    nodegraph3_node1 --"subsurface_color_out"--> nodegraph3_subsurface_color_out
    nodegraph3_node1 --"subsurface_radius_out"--> nodegraph3_subsurface_radius_out
    nodegraph3_node1 --"subsurface_scale_out"--> nodegraph3_subsurface_scale_out
    nodegraph3_node1 --"subsurface_anisotropy_out"--> nodegraph3_subsurface_anisotropy_out
    nodegraph3_node1 --"sheen_out"--> nodegraph3_sheen_out
    nodegraph3_node1 --"sheen_color_out"--> nodegraph3_sheen_color_out
    nodegraph3_node1 --"sheen_roughness_out"--> nodegraph3_sheen_roughness_out
    nodegraph3_node1 --"coat_out"--> nodegraph3_coat_out
    nodegraph3_node1 --"coat_color_out"--> nodegraph3_coat_color_out
    nodegraph3_node1 --"coat_roughness_out"--> nodegraph3_coat_roughness_out
    nodegraph3_node1 --"coat_anisotropy_out"--> nodegraph3_coat_anisotropy_out
    nodegraph3_node1 --"coat_IOR_out"--> nodegraph3_coat_IOR_out
    nodegraph3_node1 --"coat_affect_roughness_out"--> nodegraph3_coat_affect_roughness_out
    nodegraph3_node1 --"thin_film_thickness_out"--> nodegraph3_thin_film_thickness_out
    nodegraph3_node1 --"thin_film_IOR_out"--> nodegraph3_thin_film_IOR_out
    nodegraph3_node1 --"emission_out"--> nodegraph3_emission_out
    nodegraph3_node1 --"emission_color_out"--> nodegraph3_emission_color_out
    nodegraph3_node1 --"opacity_out"--> nodegraph3_opacity_out
    nodegraph3_node1 --"thin_walled_out"--> nodegraph3_thin_walled_out
```

In [517]:
from MaterialX import PyMaterialXGenShader as mx_gen_shader

target_bxdf = "standard_surface"
translator = mx_gen_shader.ShaderTranslator.create()
try:
    translator.translateAllMaterials(doc, target_bxdf )
except mx.Exception as err:
    print(err)



## Versioning

At time of writing there is no support for translating between different versions of the same BXDF shader model.
One possible proposal is to add a `source_version` and `target_version` attribute to the translator nodedefs to allow for this functionality in the future. 

e.g.

```xml
 <nodedef name="ND_open_pbr_surface_surfaceshader" node="open_pbr_surface" nodegroup="pbr" isdefaultversion="true"
           doc="OpenPBR Surface Shading Model" uiname="OpenPBR Surface">
 ```
 would become
 
 ```xml
    <nodedef name="ND_open_pbr_surface_surfaceshader" node="open_pbr_surface" nodegroup="pbr" isdefaultversion="true"   source_version="1.0" target_version="1.1"
            doc="OpenPBR Surface Shading Model" uiname="OpenPBR Surface">
```



## Creaing New Translators

There are currently no tools or APIs to assist in creating new translators.
